In [1]:
import os
os.chdir("../..")

In [2]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent
from shell.fleet.data.data_utilize import *
from shell.fleet.data.recv import *

from sklearn.manifold import TSNE
from torchvision.utils import make_grid
import logging
import copy
logging.basicConfig(level=logging.INFO)

In [3]:
seed_everything(0)

In [4]:
use_contrastive = True
num_tasks = 5

# num_init_tasks = 3
num_init_tasks = 4
dataset_name = "mnist"
# num_epochs = 40
num_epochs = 100

data_cfg = {
    "dataset_name": dataset_name,
    "num_tasks": num_tasks,
    "num_train_per_task": 128,
    "num_val_per_task": 102,
    'remap_labels': True,
    'use_contrastive': use_contrastive,
}
receiver_dataset = get_dataset(**data_cfg)

INFO:root:Class sequence: [5 0 3 4 2 6 7 1 8 9]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28)


In [5]:
seed_everything(20)
sender_dataset = get_dataset(**data_cfg)

INFO:root:Class sequence: [1 0 3 7 5 2 4 9 6 8]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28)


In [6]:
net_cfg = {
    'depth': num_init_tasks,
    'layer_size': 64,
    'num_init_tasks': num_init_tasks,
    'i_size': 28,
    'num_classes': 2,
    'num_tasks': num_tasks,
    'dropout': 0.0,
}

agent_cfg = {
    'memory_size': 64,
    'use_contrastive': use_contrastive,
    'save_dir': 'test',
    "dataset_name": dataset_name,
}


NetCls = MLPSoftLLDynamic
LearnerCls = CompositionalDynamicER
sharing_cfg = DictConfig({
    "scorer": "cross_entropy",
    "num_queries": 4,
    "query_score_threshold": 0.0,
})
train_cfg = {
    # "num_epochs": 40,
    "num_epochs": num_epochs,
    "component_update_freq": num_epochs,
    "num_candidate_modules": 1,
}

receiver = Agent(0, 0, receiver_dataset,
                NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg, 
                sharing_cfg)

sender_net_cfg = copy.deepcopy(net_cfg)
sender_net_cfg['num_init_tasks'] = -1
sender_net_cfg["init_ordering_mode"] = "uniform"

sender = Agent(0, 1, sender_dataset,
                NetCls, LearnerCls, sender_net_cfg, agent_cfg, train_cfg, 
                sharing_cfg)


INFO:root:Agent: node_id: 0, seed: 0
INFO:root:Agent: node_id: 0, seed: 1


In [7]:
receiver.net.structure

ParameterList(
    (0): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
    (1): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
    (2): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
    (3): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
    (4): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
)

In [8]:
for t in range(num_init_tasks):
    receiver.train(t)

INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.69224913	acc: 0.524
INFO:root:	task: avg	loss: 0.69224913	acc: 0.524
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.69224913	acc: 0.524
INFO:root:	task: avg	loss: 0.69224913	acc: 0.524
INFO:root:final components: 4
INFO:root:epochs: 0, training task: 1
INFO:root:	task: 0	loss: 0.69224913	acc: 0.524
INFO:root:	task: 1	loss: 0.69401731	acc: 0.493
INFO:root:	task: avg	loss: 0.69313322	acc: 0.508
INFO:root:epochs: 0, training task: 1
INFO:root:	task: 0	loss: 0.69224913	acc: 0.524
INFO:root:	task: 1	loss: 0.69401731	acc: 0.493
INFO:root:	task: avg	loss: 0.69313322	acc: 0.508
INFO:root:final components: 4
INFO:root:epochs: 0, training task: 2
INFO:root:	task: 0	loss: 0.69224913	acc: 0.524
INFO:root:	task: 1	loss: 0.69401731	acc: 0.493
INFO:root:	task: 2	loss: 0.69849413	acc: 0.481
INFO:root:	task: avg	loss: 0.69492019	acc: 0.499
INFO:root:epochs: 0, training task: 2
INFO:root:	task: 0	loss: 0.69224913	acc: 0.5

In [9]:
sender.net.load_state_dict(receiver.net.state_dict())

<All keys matched successfully>

In [10]:
## test on receiver dataset to make sure that net is loaded correctly

In [11]:
receiver.train_kwargs['num_candidate_modules'] = 2
receiver.train(num_init_tasks)

INFO:root:epochs: 0, training task: 4
INFO:root:	task: 0	loss: 0.39828512	acc: 0.972
INFO:root:	task: 1	loss: 0.41908697	acc: 0.983
INFO:root:	task: 2	loss: 0.38912266	acc: 0.977
INFO:root:	task: 3	loss: 0.34484556	acc: 0.986
INFO:root:	task: 4	loss: 0.69320738	acc: 0.491
INFO:root:	task: avg	loss: 0.44890954	acc: 0.882


NUM_CANDIDATE_MODULES 2 len(module_list) 0


INFO:root:epochs: 1, training task: 4
INFO:root:	task: 0	loss: 0.39828512	acc: 0.972
INFO:root:	task: 1	loss: 0.41908697	acc: 0.983
INFO:root:	task: 2	loss: 0.38912266	acc: 0.977
INFO:root:	task: 3	loss: 0.34484556	acc: 0.986
INFO:root:	task: 4	loss: 0.69357603	acc: 0.491
INFO:root:	task: avg	loss: 0.44898327	acc: 0.882
INFO:root:epochs: 2, training task: 4
INFO:root:	task: 0	loss: 0.39828512	acc: 0.972
INFO:root:	task: 1	loss: 0.41908697	acc: 0.983
INFO:root:	task: 2	loss: 0.38912266	acc: 0.977
INFO:root:	task: 3	loss: 0.34484556	acc: 0.986
INFO:root:	task: 4	loss: 0.69370434	acc: 0.491
INFO:root:	task: avg	loss: 0.44900893	acc: 0.882
INFO:root:epochs: 3, training task: 4
INFO:root:	task: 0	loss: 0.39828512	acc: 0.972
INFO:root:	task: 1	loss: 0.41908697	acc: 0.983
INFO:root:	task: 2	loss: 0.38912266	acc: 0.977
INFO:root:	task: 3	loss: 0.34484556	acc: 0.986
INFO:root:	task: 4	loss: 0.69353753	acc: 0.491
INFO:root:	task: avg	loss: 0.44897557	acc: 0.882
INFO:root:epochs: 4, training task

In [ ]:
# sender.train(0)

In [ ]:
## NOTE: BUG: where are INFO:root:candidate 5: W/update: 0.9901960784313726, WO/update: 0.9803921568627451
## INFO:root:candidate 6: W/update: 0.9901960784313726, WO/update: 0.9803921568627451
## candidate 5 and 6 has the exact same accuracy for some reasons

In [ ]:
# sender.net.components[-1].bias

In [ ]:
# # send the last component of sender to receiver
# # receiver.train_kwargs['module_list'] = [sender.net.components[-1]]
# receiver.train_kwargs['num_candidate_modules'] = 2
# receiver.train_kwargs['remove_modules'] = False
# receiver.train(num_tasks-1)

In [ ]:
# receiver.net.components[-1].bias

In [ ]:
# receiver.net.components[-2].bias

In [ ]:
# receiver.net.structure[-1]

In [ ]:
# valloader = torch.utils.data.DataLoader(receiver.dataset.valset[num_tasks-1],
#                                                 batch_size=256,
#                                                 shuffle=False,
#                                                 num_workers=4,
#                                                 pin_memory=True,
#                                                 )

# perfs, Ls = receiver.agent.conditionally_add_module(valloader, task_id=num_tasks-1)

In [ ]:
# print(perfs)
# print(Ls)